<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Formula" data-toc-modified-id="Formula-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Formula</a></span><ul class="toc-item"><li><span><a href="#Debug-Params-(Last-Month)" data-toc-modified-id="Debug-Params-(Last-Month)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Debug Params (Last Month)</a></span></li></ul></li><li><span><a href="#PLOT-REGISTRATION" data-toc-modified-id="PLOT-REGISTRATION-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>PLOT REGISTRATION</a></span><ul class="toc-item"><li><span><a href="#Maps" data-toc-modified-id="Maps-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Maps</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Summary</a></span></li></ul></li><li><span><a href="#Weather" data-toc-modified-id="Weather-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Weather</a></span></li><li><span><a href="#SOIL" data-toc-modified-id="SOIL-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>SOIL</a></span></li><li><span><a href="#Plant-Health" data-toc-modified-id="Plant-Health-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plant Health</a></span><ul class="toc-item"><li><span><a href="#General-Condition" data-toc-modified-id="General-Condition-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>General Condition</a></span></li><li><span><a href="#Plant-Nodes" data-toc-modified-id="Plant-Nodes-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Plant Nodes</a></span></li><li><span><a href="#Plant-Leaf" data-toc-modified-id="Plant-Leaf-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Plant Leaf</a></span></li><li><span><a href="#Plant-Branch" data-toc-modified-id="Plant-Branch-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Plant Branch</a></span></li><li><span><a href="#Other-Information" data-toc-modified-id="Other-Information-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Other Information</a></span></li></ul></li><li><span><a href="#SEND-EMAIL" data-toc-modified-id="SEND-EMAIL-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>SEND EMAIL</a></span></li></ul></div>

In [1]:
from datetime import datetime,timedelta, date
from pytz import utc, timezone
import time
from Akvo import Flow
from FlowHandler import FlowHandler
import pandas as pd
import json
import os
from jinja2 import Environment, FileSystemLoader
from IPython.core.display import display, HTML

## Formula

**Nitrate**

- N = (x + 0.613) * DF / 2.7763
- DF = (1 + (SM * 1.3)) * 5
- SM is Soil Moisture

--------------

Phospate N/A not Zero
Mean only recorded values

In [2]:
import smtplib
import sys
from email import encoders
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [3]:
instanceURI = 'spiceup'
requestURI = 'https://api.akvo.org/flow/orgs/' + instanceURI
EMAIL_RECEPIENTS = ['mail@dedenbangkit.com','ima@akvo.org','joy@akvo.org']
EMAIL_SENDER = os.environ['G4AW_EMAIL']
EMAIL_PASSWORD = os.environ['G4AW_PWD']

In [4]:
apiData = Flow.getResponse(requestURI + '/surveys/227030237')
#apiData = Flow.getResponse(requestURI + '/surveys/24210001')
forms = apiData.get('forms')
RegistrationForm = apiData['registrationFormId']
start_time = time.time()
date_format = '%Y-%m-%dT%H:%M:%SZ'

In [5]:
today_date = datetime.today().date()
week_ago = today_date - timedelta(days=30)

In [6]:
def formatDate(dt):
    return dt.strftime("%d/%m/%Y")

In [7]:
def checkTime(x):
    total_time = x - start_time
    spent = time.strftime("%H:%M:%S", time.gmtime(total_time))
    return spent

In [8]:
questions = lambda x : [{'id':a['id'],'name':a['name'],'questions':details(a['questions'])} for a in x]
details = lambda x : [{'id':a['id'],'name':a['name'].replace(' ','_'),'type':a['type']} for a in x]

In [9]:
registrationFormId = apiData.get('registrationFormId')

In [10]:
def getAll(url):
    data = Flow.getResponse(url)
    formInstances = data.get('formInstances')
    for dataPoint in formInstances:
        dataPoints.append(dataPoint)
    try:
        print(checkTime(time.time()) + ':: GET DATA FROM[' + url + ']')
        url = data.get('nextPageUrl')
        getAll(url)
    except:
        #print(checkTime(time.time()) + ':: DOWNLOAD COMPLETE')
        return "done"

In [11]:
allResponses = {}
debugArray = []
for form in forms:
    questionGroups = questions(form['questionGroups'])
    metas = pd.DataFrame(questionGroups)
    formURI = form['formInstancesUrl']
    allGroups = {}
    for index, questionGroup in enumerate(questionGroups):
        dataPoints = []
        groupID = questionGroup['id']
        metadata = metas['questions'][index]
        print(metadata)
        getAll(formURI)
        output = pd.DataFrame(dataPoints)
        debugArray.extend(dataPoints)
        #print(checkTime(time.time()) + ':: TRANSFORMING')
        for qst in metadata:
            qName = qst['name'].replace('_',' ')
            qId = str(qst['id'])
            qType = qst['type']
            try: 
                output[qName] = output['responses'].apply(lambda x: FlowHandler(x.get(groupID, False),qId,qType))
                if qType == 'GEO':
                    output[qName+'_lat'] = output[qName].apply(lambda x: x[0] if x is not None else x)
                    output[qName+'_long'] = output[qName].apply(lambda x: x[1] if x is not None else x)
                    output = output.drop([qName], axis=1)
            except:
                pass
        try:
            output = output.drop(['responses'], axis=1)
        except:
            pass
        allGroups.update({questionGroup['name']:output.to_dict('records')})
    allResponses.update({form['name']:allGroups})

[{'id': '224750001', 'name': 'Soil_ID', 'type': 'FREE_TEXT'}, {'id': '251820002', 'name': 'Soil_Moisture', 'type': 'CADDISFLY'}, {'id': '250990212', 'name': 'Soil_pH', 'type': 'CADDISFLY'}, {'id': '240920193', 'name': 'Nitrate', 'type': 'CADDISFLY'}, {'id': '240920199', 'name': 'P', 'type': 'CADDISFLY'}]
00:00:07:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=227030238]
00:00:22:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=227030238&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGLGty3kMGAAgAA]
00:00:25:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=227030238&cursor=CjASKmoOZX5ha3ZvZmxvdy0xNzdyGAsSDlN1cnZleUluc3RhbmNlGPXp-4UBDBgAIAA]
00:00:27:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=227030238&cursor=CjASKmoOZX5ha3ZvZmxvdy0xNzdyGAsSDlN1cnZleUluc3RhbmNlGJHR1YoBDBgAIAA

00:02:36:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=CjASKmoOZX5ha3ZvZmxvdy0xNzdyGAsSDlN1cnZleUluc3RhbmNlGIaFiocBDBgAIAA]
00:02:39:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=CjASKmoOZX5ha3ZvZmxvdy0xNzdyGAsSDlN1cnZleUluc3RhbmNlGJfY-IgBDBgAIAA]
00:02:40:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=CjASKmoOZX5ha3ZvZmxvdy0xNzdyGAsSDlN1cnZleUluc3RhbmNlGNyI44oBDBgAIAA]
[{'id': '217070156', 'name': 'Plant_Height_(cm)', 'type': 'NUMBER'}, {'id': '235080148', 'name': 'Number_of_Nodes', 'type': 'NUMBER'}, {'id': '227030242', 'name': 'Number_of_Branches', 'type': 'NUMBER'}, {'id': '240920201', 'name': 'Number_of_Leaves', 'type': 'NUMBER'}, {'id': '247060418', 'name': 'Number_of_Orthotropic_shoots', 'type': 'NUMBER'}]
00:02:45:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_in

00:04:46:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331]
00:04:49:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGOGAlmYMGAAgAA]
00:04:52:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGKvi_WYMGAAgAA]
00:04:55:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGKvG_mcMGAAgAA]
00:04:59:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGKrr-WgMGAAgAA]
00:05:02:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=2470503

00:07:10:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGKOg8moMGAAgAA]
00:07:13:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGLeUkGsMGAAgAA]
00:07:16:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGIGw72sMGAAgAA]
00:07:18:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGI_V6mwMGAAgAA]
00:07:21:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGLyd6m0MGAAgAA]
00:07:24:: GET DATA FROM[https://api.akv

[{'id': '240920204', 'name': 'General_Weather_Conditions', 'type': 'CASCADE'}, {'id': '251000133', 'name': 'Temperature_(Celsius)', 'type': 'NUMBER'}, {'id': '221090027', 'name': 'Time', 'type': 'CASCADE'}, {'id': '247170193', 'name': 'Comment', 'type': 'FREE_TEXT'}]
00:08:43:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=235100158]
00:08:46:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=235100158&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGIL_4moMGAAgAA]
00:08:48:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=235100158&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGOGkwG4MGAAgAA]
00:08:51:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=235100158&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGJLjlHQMGAAgAA]
00:08:53:: GET DATA FROM[https://api.a

In [12]:
pd.DataFrame(allResponses).unstack().to_frame().dropna().reset_index()

level_0              level_1  \
0     03. Soil Monitoring      Soil Properties   
1   01. Plot Registration   Plant Registration   
2    04. Plant Monitoring    General Condition   
3    04. Plant Monitoring    Other Information   
4    04. Plant Monitoring         Plant Growth   
5    04. Plant Monitoring         Plant Health   
6            05. Activity  Activity Monitoring   
7  02. Weather Conditions   Weather Conditions   

                                                   0  
0  [{'createdAt': '2019-10-17T03:06:38.966Z', 'da...  
1  [{'createdAt': '2019-04-16T10:20:17.342Z', 'da...  
2  [{'createdAt': '2019-08-27T12:46:48.633Z', 'da...  
3  [{'createdAt': '2019-08-27T12:46:48.633Z', 'da...  
4  [{'createdAt': '2019-08-27T12:46:48.633Z', 'da...  
5  [{'createdAt': '2019-08-27T12:46:48.633Z', 'da...  
6  [{'createdAt': '2020-01-27T07:09:29.817Z', 'da...  
7  [{'createdAt': '2019-04-20T01:04:12.874Z', 'da...

In [13]:
pd.DataFrame(questionGroups)

id                name  \
0  213030199  Weather Conditions   

                                           questions  
0  [{'id': '240920204', 'name': 'General_Weather_...

In [14]:
qGroup = list(allResponses.keys())
qGroup.sort()

In [15]:
#allResponses[qGroup[4]]
allResponses['02. Weather Conditions']['Weather Conditions']

[{'createdAt': '2019-04-20T01:04:12.874Z',
  'dataPointId': '245520001',
  'deviceIdentifier': 'CAN',
  'displayName': 'Demo Plot Bangka - Kepulauan Bangka Belitung - Kab. Bangka Tengah - Namang - Desa Namang',
  'formId': '235100158',
  'id': '213640002',
  'identifier': '23jy-48c7-2pax',
  'modifiedAt': '2019-04-20T01:04:15.461Z',
  'submissionDate': '2019-04-20T01:03:35Z',
  'submitter': 'saepul',
  'surveyalTime': 11,
  'General Weather Conditions': ':Thick Cloudy',
  'Temperature (Celsius)': 28.0,
  'Time': 'Pagi',
  'Comment': None},
 {'createdAt': '2019-04-22T02:23:43.653Z',
  'dataPointId': '245520001',
  'deviceIdentifier': 'CAN',
  'displayName': 'Demo Plot Bangka - Kepulauan Bangka Belitung - Kab. Bangka Tengah - Namang - Desa Namang',
  'formId': '235100158',
  'id': '213670001',
  'identifier': '23jy-48c7-2pax',
  'modifiedAt': '2019-04-22T02:23:48.034Z',
  'submissionDate': '2019-04-22T02:23:07Z',
  'submitter': 'saepul',
  'surveyalTime': 12,
  'General Weather Condition

In [16]:
newResponses = {}
newSort = sorted(allResponses.keys())
for ns in newSort:
    newResponses.update({ns:allResponses[ns]})
    
qGroup

['01. Plot Registration',
 '02. Weather Conditions',
 '03. Soil Monitoring',
 '04. Plant Monitoring',
 '05. Activity']

In [17]:
for g in qGroup:
    forms = [k for k in newResponses[g]]
    for form in forms:
        unordered = pd.DataFrame(newResponses[g][form])
        #print(form)
        #print(list(unordered))
        try:
            ordered = unordered.sort_values(by='submissionDate',ascending=False).to_dict('records')
            newResponses[g][form] = ordered
        except:
            pass
allResponses = newResponses

In [18]:
#this_month = datetime.now().strftime('%B')

### Debug Params (Last Month)

In [19]:
this_month = (today_date - timedelta(days=30)).strftime('%B')

In [20]:
email_data = {}

## PLOT REGISTRATION

In [21]:
plant = allResponses['01. Plot Registration']['Plant Registration'][0]
plot_location = plant['Plot Location'].split('|')
plot_location = [pl.split(':')[0] for pl in plot_location]
plot_location = ', '.join(plot_location)
plot_location

'Kepulauan Bangka Belitung, Kab. Bangka TengahKab. Bangka Tengah, NamangNamang, Desa NamangDesa Namang'

In [22]:
email_data.update({'plot_location':plot_location})

### Maps

In [23]:
plot_latlong = [str(plant['Plot Geopoint_lat']),str(plant['Plot Geopoint_long'])]
plot_latlong = ', '.join(plot_latlong)
plot_map_img = "https://www.mapquestapi.com/staticmap/v4/getmap?size=640,480&type=map&pois=mcenter," + plot_latlong
plot_map_img = plot_map_img +"|&zoom=12&center="+ plot_latlong +"&imagetype=JPEG&traffic=flow&scalebar=false&key=lYrP4vF3Uk5zgTiGGuEzQGwGIVDGuy24"

In [24]:
plot_link = "http://maps.google.com/maps?t=k&q=loc:" + plot_latlong

In [25]:
plot_link

'http://maps.google.com/maps?t=k&q=loc:-2.3428743096598823, 106.18896435946226'

In [26]:
email_data.update({'plot_link':plot_link})

### Summary

In [27]:
plant_submissionDate = plant['submissionDate'].split('T')[0]
plot_submission_summary = 'First demonstration plots have been planted at '
plot_submission_summary =  plot_submission_summary + datetime.strptime(plant_submissionDate, '%Y-%m-%d').strftime('%B, %d')
plant_type = plant['Type of Plantation']
plot_submission_summary = plot_submission_summary + ' with ' + plant_type + ' in ' + plot_location + '.'

In [28]:
plot_submission_summary

'First demonstration plots have been planted at April, 16 with White Pepper in Kepulauan Bangka Belitung, Kab. Bangka TengahKab. Bangka Tengah, NamangNamang, Desa NamangDesa Namang.'

In [29]:
email_data.update({'plot_submission_summary':plot_submission_summary})

## Weather

In [30]:
weather = pd.DataFrame(allResponses['02. Weather Conditions']['Weather Conditions'])
weather['General Weather Conditions'] = weather['General Weather Conditions'].apply(lambda x: x.replace(':',''))
weather['createdAt'] = weather['createdAt'].astype('datetime64[ns]')
weather['month'] = weather['createdAt'].apply(lambda x : x.strftime('%B'))
#weather = weather[weather['month'] == this_month]
try:
    weather_average_temperature = list(weather.groupby('identifier').mean().reset_index()['Temperature (Celsius)'])[0]
    weather_average_temperature = str(int(weather_average_temperature)) + '° Celcious'
except:
    weather_averrage_temperature = 'nan'
weather_average_temperature

'28° Celcious'

In [31]:
email_data.update({'weather_average_temperature':weather_average_temperature})

In [32]:
weather_most_condition = weather.groupby(['General Weather Conditions','createdAt']).size().to_frame('size').reset_index()
weather_most_condition = weather_most_condition.groupby(['General Weather Conditions']).size().to_frame('size').reset_index()
weather_most_condition = weather_most_condition.max()[0]
try:
    weather_most_condition = 'Mostly ' + weather_most_condition
except:
    weather_most_condition = 'nan'

In [33]:
weather_most_condition

'Mostly Thick Cloudy'

In [34]:
email_data.update({'weather_most_condition':weather_most_condition})

In [35]:
weather_all_conditions = weather[['dataPointId','General Weather Conditions']].groupby(['General Weather Conditions']).count().reset_index()

In [36]:
email_data.update({'weather_all_conditions':weather_all_conditions.to_dict('records')})

## SOIL

In [37]:
soil = pd.DataFrame(allResponses['03. Soil Monitoring']['Soil Properties'])
soil['createdAt'] = soil['createdAt'].astype('datetime64[ns]')
soil['month'] = soil['createdAt'].apply(lambda x : x.strftime('%B'))
soil = soil[soil['month'] == this_month]

In [38]:
def get_soil_result(x):
    if x is not None:
        return float(x['result'][0]['value'])
    else:
        return 0

In [39]:
soil['Soil Moisture'] = soil['Soil Moisture'].apply(get_soil_result)
soil['Soil pH'] = soil['Soil pH'].apply(get_soil_result)
soil['Nitrate'] = soil['Nitrate'].apply(get_soil_result)
soil['P'] = soil['P'].apply(get_soil_result)

In [40]:
soil_average = soil.groupby('identifier').mean().reset_index()
soil_p = round(soil_average['P'], 2)
soil_ph = round(soil_average['Soil pH'], 2)
soil_nitrate = round(soil_average['Nitrate'][0], 2)
soil_moisture = round(soil_average['Soil Moisture'][0], 2)
soil_df = (1 + (soil_moisture * 1.3)) * 5
soil_formula = round((soil_nitrate + 0.613) * soil_df / 2.7763, 2)
soil_formula

135.73

In [41]:
pd.DataFrame(allResponses['03. Soil Monitoring']['Soil Properties'])

Nitrate     P Soil ID  \
0   {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NB2   
1   {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NB1   
2   {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NA2   
3   {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NAI   
4   {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PB2   
5   {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PB1   
6   {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PA2   
7   {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PA1   
8                                                None  None     NA2   
9                                                None  None     NB2   
10                                               None  None     NB1   
11                                               None  None     NA1   
12                                               None  None     PB2   
13                                               None  None     PB1   
14                                               None  None     PA1   
15                                               None  None     PA2   
16  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NB2   
17  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NB1   
18  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NA2   
19  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NA1   
20  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PB2   
21  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PB1   
22  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PA2   
23  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PA1   
24                                               None  None     NB2   
25                                               None  None     NB1   
26                                               None  None     NA2   
27                                               None  None     NA1   
28                                               None  None     PB2   
29                                               None  None     PB1   
..                                                ...   ...     ...   
40                                               None  None     NB2   
41                                               None  None     NB1   
42                                               None  None     NA1   
43                                               None  None     NA2   
44                                               None  None     PB1   
45                                               None  None     PA1   
46                                               None  None     PA2   
47                                               None  None     PB2   
48  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NB2   
49  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NB1   
50  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NA2   
51  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     NA1   
52  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PB2   
53  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PB1   
54  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PA2   
55  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None     PA1   
56                                               None  None     PA2   
57                                               None  None     NB2   
58                                               None  None     NB1   
59                                               None  None     NA2   
60                                               None  None     NA1   
61                                               None  None     PB2   
62                                               None  None     PB1   
63                                               None  None     PA1   
64  {'type': 'caddisfly', 'name': 'Water - Nitrate...  None    None   

In [42]:
#Flow.getResponse('https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331')

## Plant Health

In [43]:
planthealth = pd.DataFrame(allResponses['04. Plant Monitoring'])

### General Condition

In [44]:
plant_gc = pd.DataFrame(allResponses['04. Plant Monitoring']['General Condition'])
plant_gc

Plant / Vine Condition Plant Condition Plant ID Roots / Stem Base  \
0                 Vigourous           Alive      N55           Whitish   
1                 Vigourous           Alive      N56           Whitish   
2                 Vigourous           Alive      N54           Whitish   
3                      None            Dead      N57           Whitish   
4                 Vigourous           Alive      N53           Whitish   
5                 Vigourous           Alive      N49           Whitish   
6                 Vigourous           Alive      N52           Whitish   
7                 Vigourous           Alive      N50           Whitish   
8                 Vigourous           Alive      N51           Whitish   
9                 Vigourous           Alive      N46           Whitish   
10                Vigourous           Alive      N47           Whitish   
11                Vigourous           Alive      N45           Whitish   
12                Vigourous           Alive      N48           Whitish   
13                     None            Dead      N40           Whitish   
14                Vigourous           Alive      N43           Whitish   
15                Vigourous           Alive      N41           Whitish   
16                     None            Dead      N42           Whitish   
17                Vigourous           Alive      N37           Whitish   
18                Vigourous           Alive      N38           Whitish   
19                     None            Dead      N36           Whitish   
20                Vigourous           Alive      N39           Whitish   
21                Vigourous           Alive      N35           Whitish   
22                Vigourous           Alive      N31           Whitish   
23                Vigourous           Alive      N34           Whitish   
24                Vigourous           Alive      N32           Whitish   
25                Vigourous           Alive      N33           Whitish   
26                Vigourous           Alive      N28           Whitish   
27                Vigourous           Alive      N29           Whitish   
28                Vigourous           Alive      N30           Whitish   
29                Vigourous           Alive      N26           Whitish   
...                     ...             ...      ...               ...   
1039                   None            None      N30              None   
1040                   None            None      N29              None   
1041                   None            None      N28              None   
1042                   None            None      N27              None   
1043                   None            None      N26              None   
1044                   None            None      N25              None   
1045                   None            None      N24              None   
1046                   None            None      N23              None   
1047                   None            None      N22              None   
1048                   None            None      N21              None   
1049                   None            None      N20              None   
1050                   None            None      N19              None   
1051                   None            None      N18              None   
1052                   None            None      N17              None   
1053                   None            None      N16              None   
1054                   None            None      N15              None   
1055                   None            None      N14              None   
1056                   None            None      N13              None   
1057                   None            None      N12              None   
1058                   None            None      N11              None   
1059                   None            None      N10              None   
1060                   None            None       N9              None   
1061     

In [45]:
plant_gc['createdAt'] = plant_gc['createdAt'].astype('datetime64[ns]')
plant_gc['month'] = plant_gc['createdAt'].apply(lambda x : x.strftime('%B'))
#plant_gc = plant_gc[plant_gc['month'] == this_month]

In [46]:
plant_height_max = plant_gc.max()
plant_height_min = plant_gc.min()
plant_height_avg = plant_gc.mean()

In [47]:
variables = ['Plant / Vine Condition','Roots / Stem Base']
conditions = plant_gc[variables + ['Plant ID']].groupby(variables).count().reset_index().to_dict('records')
plant_general_condition = []
for condition in conditions:
    plant_general_condition.append(str(condition['Plant ID']) + " plants exhibited " + condition['Roots / Stem Base'].lower() + " roots and " + condition['Plant / Vine Condition'].lower() + " vine condition")

plant_general_condition = ', '.join(plant_general_condition)
plant_general_condition

'574 plants exhibited whitish roots and vigourous vine condition, 34 plants exhibited whitish roots and wilt vine condition, 2 plants exhibited whitish roots and yellowing vine condition'

### Plant Nodes

In [48]:
plant_growth = pd.DataFrame(allResponses['04. Plant Monitoring']['Plant Growth'])
plant_growth

Number of Branches  Number of Leaves  Number of Nodes  \
0                    1.0              28.0              3.0   
1                    1.0              26.0              4.0   
2                    4.0              43.0              6.0   
3                    NaN               NaN              NaN   
4                    3.0              70.0              4.0   
5                    0.0               1.0              7.0   
6                    2.0              44.0              2.0   
7                    1.0              35.0              5.0   
8                    1.0              22.0              5.0   
9                   26.0             247.0             10.0   
10                   2.0              69.0              6.0   
11                   1.0              15.0              4.0   
12                   1.0              18.0              6.0   
13                   NaN               NaN              NaN   
14                   5.0              64.0              4.0   
15                   0.0               3.0              5.0   
16                   NaN               NaN              NaN   
17                   5.0              52.0             26.0   
18                   0.0               0.0              1.0   
19                   NaN               NaN              NaN   
20                   4.0              30.0              8.0   
21                   7.0              52.0              4.0   
22                   5.0             157.0              9.0   
23                   3.0              25.0              8.0   
24                   7.0              67.0              9.0   
25                   2.0              37.0              6.0   
26                   1.0               2.0              1.0   
27                   4.0              26.0              9.0   
28                   4.0              56.0              8.0   
29                  10.0             125.0             15.0   
...                  ...               ...              ...   
1039                 4.0              35.0             21.0   
1040                 5.0              17.0             16.0   
1041                 6.0              52.0             23.0   
1042                 7.0              46.0             22.0   
1043                 6.0              47.0             20.0   
1044                 5.0              39.0             13.0   
1045                 4.0              31.0             18.0   
1046                 5.0              38.0             16.0   
1047                 4.0              36.0             21.0   
1048                 5.0              44.0             18.0   
1049                 7.0              34.0             15.0   
1050                 3.0              22.0             14.0   
1051                 4.0              38.0             24.0   
1052                 3.0              29.0             16.0   
1053                 3.0              18.0             11.0   
1054                 6.0              45.0             17.0   
1055                 2.0              16.0             16.0   
1056                 4.0              27.0             18.0   
1057                 8.0              74.0             24.0   
1058                 8.0              60.0             20.0   
1059                 6.0              48.0             17.0   
1060                11.0              95.0             25.0   
1061                 9.0              70.0             26.0   
1062                 8.0              57.0             18.0   
1063                 2.0              37.0             15.0   
1064                 4.0              36.0             17.0   
1065                 7.0              68.0             25.0   
1066                 4.0              35.0             22.0   
1067                 8.0              64.0             21.0   
1068                 4.0              34.0             18.0   

      Number of Orthotropic shoots  Plant Height (cm)  \
0                              1.0        

In [49]:
plant_growth['createdAt'] = plant_growth['createdAt'].astype('datetime64[ns]')
plant_growth['month'] = plant_growth['createdAt'].apply(lambda x : x.strftime('%B'))
#plant_growth = plant_growth[plant_growth['month'] == this_month]

In [50]:
#plant_node_max = plant_node.max()
#plant_node_min = plant_node.min()
#plant_node_avg = plant_node.mean()

In [51]:
plant_node = plant_growth['Number of Nodes'].describe()

In [52]:
plant_node

count    913.000000
mean      15.018620
std       12.184919
min        0.000000
25%        9.000000
50%       15.000000
75%       20.000000
max      307.000000
Name: Number of Nodes, dtype: float64

### Plant Leaf

In [53]:
#plant_leaf = pd.DataFrame(allResponses['04. Plant Monitoring']['Plant Growth'])
#plant_leaf['createdAt'] = plant_leaf['createdAt'].astype('datetime64[ns]')
#plant_leaf['month'] = plant_leaf['createdAt'].apply(lambda x : x.strftime('%B'))
#plant_leaf = plant_leaf[plant_leaf['month'] == this_month]
#plant_leaf_max = plant_leaf.max()
#plant_leaf_min = plant_leaf.min()
#plant_leaf_avg = plant_leaf.mean()

plant_leaf = plant_growth['Number of Leaves'].describe()
plant_leaf

count    920.000000
mean      46.027174
std       35.860877
min        0.000000
25%       21.000000
50%       37.000000
75%       61.000000
max      276.000000
Name: Number of Leaves, dtype: float64

In [54]:
plant_leaf

count    920.000000
mean      46.027174
std       35.860877
min        0.000000
25%       21.000000
50%       37.000000
75%       61.000000
max      276.000000
Name: Number of Leaves, dtype: float64

### Plant Branch

In [55]:
#plant_branch = pd.DataFrame(allResponses['04. Plant Monitoring']['Plant Branch'])
#plant_branch['createdAt'] = plant_branch['createdAt'].astype('datetime64[ns]')
#plant_branch['month'] = plant_branch['createdAt'].apply(lambda x : x.strftime('%B'))
#plant_branch = plant_branch[plant_branch['month'] == this_month]
#plant_branch_max = plant_branch.max()
#plant_branch_min = plant_branch.min()
#plant_branch_avg = plant_branch.mean()

plant_branch = plant_growth['Number of Branches'].describe()
plant_branch

count    920.000000
mean       6.004348
std        4.748745
min        0.000000
25%        3.000000
50%        5.000000
75%        8.000000
max       80.000000
Name: Number of Branches, dtype: float64

In [56]:
plant_height = plant_growth['Plant Height (cm)'].describe()
plant_height

count    913.000000
mean      70.985761
std       37.181053
min        0.000000
25%       40.000000
50%       71.000000
75%       95.000000
max      221.000000
Name: Plant Height (cm), dtype: float64

In [57]:
plant_orthotropic = plant_growth['Number of Orthotropic shoots'].describe()
plant_orthotropic

count    918.000000
mean       1.438998
std        0.873756
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       14.000000
Name: Number of Orthotropic shoots, dtype: float64

### Other Information

In [58]:
plant_oi = pd.DataFrame(allResponses['04. Plant Monitoring']['Other Information'])
plant_oi['createdAt'] = plant_oi['createdAt'].astype('datetime64[ns]')
plant_oi['month'] = plant_oi['createdAt'].apply(lambda x : x.strftime('%B'))
#plant_oi = plant_oi[plant_oi['month'] == this_month]
plant_oi_max = plant_oi.max()
plant_oi_min = plant_oi.min()
plant_oi_avg = plant_oi.mean()
plant_oi

Leaves hanging rigidly  \
0                        No   
1                        No   
2                        No   
3                      None   
4                        No   
5                        No   
6                        No   
7                        No   
8                        No   
9                        No   
10                       No   
11                       No   
12                       No   
13                     None   
14                       No   
15                       No   
16                     None   
17                       No   
18                       No   
19                     None   
20                       No   
21                       No   
22                       No   
23                       No   
24                       No   
25                       No   
26                       No   
27                       No   
28                       No   
29                       No   
...                     ...   
1039                   None   
1040                   None   
1041                   None   
1042                   None   
1043                   None   
1044                   None   
1045                   None   
1046                   None   
1047                   None   
1048                   None   
1049                   None   
1050                   None   
1051                   None   
1052                   None   
1053                   None   
1054                   None   
1055                   None   
1056                   None   
1057                   None   
1058                   None   
1059                   None   
1060                   None   
1061                   None   
1062                   None   
1063                   None   
1064                   None   
1065                   None   
1066                   None   
1067                   None   
1068                   None   

                                                  Notes  \
0                                                  None   
1                                                  None   
2                                                  None   
3                                                  None   
4                                                  None   
5                                                  None   
6                                                  None   
7                                                  None   
8                                                  None   
9                                                  None   
10                                                 None   
11                 pada daun terlihat seperti berkarat.   
12                                                 None   
13                      mati pada pengamatan sebelumnya   
14                                                 None   
15                                                 None   
16                      mati pada pengamatan sebelumnya   
17                                                 None   
18                                                 None   
19                      mati pada pengamatan sebelumnya   
20                                                 None   
21                                                 None   
22                                                 None   
23                                                 None   
24                                                 None   
25                                                 None   
26                                                 None   
27    beberapa daun rontok, pada ruas kuning kehitaman.   
28                                                 None   
29                                                 None   
...                                                 ...   
1039                                                 06   
1040                                                 06   
1041                                                 06   
1042             

In [59]:
plant_oi_hanging = plant_oi.groupby('Leaves hanging rigidly').size()
plant_oi_wilting = plant_oi.groupby('Wilting Symptoms').size()
plant_oi_wilting

Wilting Symptoms
No    200
dtype: int64

In [60]:
activities = pd.DataFrame.from_dict(allResponses['05. Activity']['Activity Monitoring'])
type_of_activities = activities['Type of Activity'].unique()

In [65]:
weather_img_mapping = {
    'Mostly Cloudy': 'https://i.imgur.com/RP7sg1j.png',
    'Mostly Light Rain': 'https://i.imgur.com/hlFuSZZ.png',
    'Mostly Sunny with Cloud': 'https://i.imgur.com/vdgWxeP.png',
    'Mostly Sunny': 'https://i.imgur.com/JD4Z8Rs.png',
    'Mostly Thick Cloudy': 'https://i.imgur.com/RP7sg1j.png',
    'Mostly Medium Rain': 'https://i.imgur.com/Yt8QleJ.png',
    'Mostly Heavy Rain': 'https://i.imgur.com/bIJdjUl.png'
}

today = date.today()

file_loader = FileSystemLoader('.')
env = Environment(loader=file_loader)
template = env.get_template('testing-real.html')
html_output = template.render(
    date = today.strftime("%B %d, %Y"),
    plot_summary = plot_submission_summary,
    img_map = 'https://www.mapquestapi.com/staticmap/v4/getmap?size=640,480&amp;type=map&amp;pois=mcenter,-2.3428743096598823, 106.18896435946226|&amp;zoom=12&amp;center=-2.3428743096598823, 106.18896435946226&amp;imagetype=JPEG&amp;traffic=flow&amp;scalebar=false&amp;key=lYrP4vF3Uk5zgTiGGuEzQGwGIVDGuy24',
    gmap_link = plot_link,
    weather_icon = weather_img_mapping.get(weather_most_condition, weather_img_mapping['Mostly Sunny']),
    weather_label = weather_most_condition,
    weather_temp = weather_average_temperature,
    soil_moisture = round(soil_average['Soil Moisture'][0], 2),
    soil_ph = round(soil_average['Soil pH'][0], 2),
    soil_nitrate = round(soil_average['Nitrate'][0], 2),
    soil_p = round(soil_average['P'][0], 2),
    plant_general_condition = plant_general_condition,
    plant_leaf = plant_leaf,
    plant_node = plant_node,
    plant_height = plant_height,
    plant_branch = plant_branch,
    plant_oi_hanging = plant_oi_hanging,
    plant_oi_wilting = plant_oi_wilting,
    plant_orthotropic = plant_orthotropic,
    type_of_activities = type_of_activities
)
display(HTML(html_output))

## SEND EMAIL

In [66]:
fout = "./testing-real.html"

msg = MIMEMultipart('alternative')
msg['Subject'] = 'TEST. DO NOT REPLY: Demo Plot Notification from ' + formatDate(week_ago) + ' to ' + formatDate(today_date)
msg['To'] = ', '.join(EMAIL_RECEPIENTS)
msg['From'] = EMAIL_SENDER

print(checkTime(time.time()) + ' SENDING EMAIL')
with open(fout) as fp:
    msg.attach(MIMEText(html_output, 'html'))

try:
    with smtplib.SMTP('smtp.gmail.com', 587) as s:
            s.ehlo()
            s.starttls()
            s.ehlo()
            s.login(EMAIL_SENDER, EMAIL_PASSWORD)
            s.send_message(msg)
            s.quit()
    print(checkTime(time.time()) + ' EMAIL SENT')
except:
    print(checkTime(time.time()) + ' UNABLE TO SEND THE EMAIL. ERROR:',sys.exc_info()[0])
    raise

00:37:06 SENDING EMAIL
00:37:09 EMAIL SENT
